# This file begins pulling senate contributions into a dataframe

In [20]:
import requests,json,pprint
import pandas as pd
import pprint
from api_keys import lda_senate_api_key

headers = {f"Authorization: Token {lda_senate_api_key}"}

url = "https://lda.senate.gov/api/v1/contributions"
response = requests.get(url,headers=headers).json()
pprint.pprint(response)

{'detail': 'Request was throttled. Expected available in 1033 seconds.'}


In [13]:
# This section does a couple things:
# 1. Begins populating new_dict_list with information we want. Later it is converted to a df
# 2. Counts how many results exist and how many contributions exist in each result to make sense 
# of what is being pulled versus what is present. Essentially, to make sure all are accounted for.
# 3. It only loops through the first three pages as a test

new_dict_list = []

# Even though the URL is already defined above, if you don't define it again here,
# it will start at the last page left off as it keeps updating the global variable with the last page.
url = "https://lda.senate.gov/api/v1/contributions/?page=1001"


# Reset cummulativeContributions
cummulativeContributions = 0

# Loop through first three pages
for i in range(7000):
    #url = f"https://lda.senate.gov/api/v1/contributions/?page={i}"
    print(url)
    response = requests.get(url,headers=headers).json()
    resultCount = 0
    totalContributions = 0
    for eachResult in response["results"]:
        contributionCount = 0
        for eachContribution in eachResult["contribution_items"]:
            
            # Store information from response as new entry in dictionary
            new_entry = {"amount":eachContribution["amount"],
                         "contributor_name":eachContribution["contributor_name"],
                         "date":eachContribution["date"],
                         "honoree_name":eachContribution["honoree_name"],
                         "state":eachResult["state"],
                         "country":eachResult["country"]}      
            new_dict_list.append(new_entry)
            contributionCount =contributionCount+1
        #print(f"Contribution count is {contributionCount}")
        resultCount =resultCount+1
        totalContributions = totalContributions + contributionCount
    print(f"Result count is {resultCount}")
    print(f"Total Contributions for this page is {totalContributions}")
    cummulativeContributions = cummulativeContributions + totalContributions
    #url = response["next"]
        
    
print(f"Cummulative contributions is {cummulativeContributions}")
df = pd.DataFrame(new_dict_list)

df.shape

https://lda.senate.gov/api/v1/contributions/?page=1001


KeyError: 'results'

In [4]:
df

,amount,contributor_name,date,honoree_name,state,country
0,500.00,"FULLER, CAROLYN",2008-03-09,Senator Daniel Inouye,VA,US
1,500.00,"FULLER, CAROLYN",2008-06-26,Congressman Mike Simpson,VA,US
2,250.00,"FULLER, CAROLYN",2008-03-09,Congressman Bill Sali,VA,US
3,2500.00,"FULLER, CAROLYN",2008-04-01,Senator Jon Tester,VA,US
4,500.00,"FULLER, CAROLYN",2008-06-25,Senator Max Baucus,VA,US
...,...,...,...,...,...,...
134087,1000.00,"HIEBERT, PETER N",2008-09-18,Senator Harry Reid,DC,US
134088,500.00,"HIEBERT, PETER N",2008-09-10,Delegate Donna Christensen,DC,US
134089,500.00,"HIEBERT, PETER N",2008-09-23,Representative Patrick Murphy,DC,US
134090,1000.00,"HIEBERT, PETER N",2008-11-17,Senator Harry Reid,DC,US


In [7]:
df.to_csv("lda_senate_output/page1-1000.csv",index=False)